# 5. Challenge Solution - REINFORCE

![cartpole](https://miro.medium.com/max/1200/1*v8KcdjfVGf39yvTpXDTCGQ.gif)

In this notebook, we're going to build a REINFORCE agent that learns to play a CartPole game. OpenAI (famously funded by Elon Musk) provides a package called "gym" that has a variety of environments where we can train our agents.

### Basic commands

We are aware that it might be your first interaction with OpenAI Gym library, therefore, we are providing you with some relevant commands you might need to use throughout both parts of the challenge.

In [1]:
def sample_commands():
    
    #Creating cartpole environment
    env = gym.make('CartPole-v0')
    
    #Visualize actions
    env.render()
    
    #Take action and extracting information
    observation, reward, done, info = env.step(action)
    
    #Close window
    env.close()
    
    #Extracting the number of possible actions
    env.action_space

### Implementation

Let's start building our agent by importing the required libraries and specifying parameters. You may play around with some of the values (gamma, learning rate, etc.) to see the effect it has on training.

In [2]:
import gym
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Input, Dense
from keras.models import Sequential

In [3]:
#####------Configuration parameters----------------------################

seed = 42
# Discount factor for past rewards
gamma = 0.99
learning_rate = 0.01
max_steps_per_episode = 10000

In [4]:
#######------Setting up environment----------------------################
env = gym.make("CartPole-v0")
env.seed(seed)
eps = np.finfo(np.float32).eps.item()  # Smallest number such that 1.0 + eps != 1.0

/Users/lukeleeai/opt/anaconda3/envs/tf2/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


### Define a model
Let's first build a model. It's a simple neural nets that takes in states as inputs and predicts which policy to follow. And that's it! You're soon going to see how we can train this simple model to predict the best policy in each state. 

In [5]:
num_states = env.observation_space.shape[0]
num_hiddens1 = 32
num_hiddens2 = 32
num_actions = env.action_space.n  # there are two actions in this game: left and right

# define a model
inputs = Input(shape=(num_states,))
fc1 = Dense(num_hiddens1, activation='relu')(inputs)
fc2 = Dense(num_hiddens2, activation='relu')(fc1)
outputs = Dense(num_actions, activation='softmax')(fc2)

model = keras.Model(inputs=inputs, outputs=outputs)

### Define a training process

REINFORCE is one of the Policy Gradient algorithms that learns to find an optimal policy that maximises the objective function: 

$$
J(\theta) = \sum_\tau P(\tau; \theta)R(\tau)
$$

where $d_{\pi}(s)$ is probability of visiting the state $s$. Since it's nearly impossible to know the exact stationary distribution of states $d_{\pi}(s)$ especially in the continuous space, we'd like to approximate the objective function. In a nutshell, the approximation represents the expected rewards with a given policy. 

Its derivative that we will use to update our model is

$$
\begin {split}
\nabla J(\theta) 
& = \mathbb{E}_{\pi} [Q_{\pi}(s,a) {\nabla}_{\theta} \ln{\pi_\theta (a|s)}]
\\ &= \mathbb{E}_{\pi} [G_t {\nabla}_{\theta} \ln{\pi_\theta (a|s)}]
\end {split}
$$

according to the [Policy Gradient Theorem](https://lilianweng.github.io/lil-log/2018/04/08/policy-gradient-algorithms.html#policy-gradient-theorem).

To train a REINFORCE agent, we have to follow the following steps:

1. Initialize the policy with random values
2. Sample states, actions, and rewards from one episode
3. Update the policy using the gradient of the objective function $J(\theta)$
4. Repeat 2~3 until the policy converges

We just assume that the returns we get from each episode when following the current policy represents the objective function fairly well. By sampling them multiple times through thousands of episodes, we will hopefully update the model in the right direction. It's basically how most of the deep learning models work.

Unfortunately, rendering the game display in Jupyter Notebook requires a complex workaround. If you want to render it, copy the entire code and run it in your favorite IDE like PyCharm and uncomment the line `env.render()`

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
action_probs_history = []  # probabilities of the selected actions for each time step
rewards_history = []  # rewards for each time step
running_reward = 0
episode_count = 0

while True:
    state = env.reset()  # reset the environment 
    episode_reward = 0
    
    # tf.GradientTape() lets us compute the gradiets of loss with respect to model weights
    with tf.GradientTape() as tape: 
        for timestep in range(1, max_steps_per_episode):
            # env.render()  # Uncomment this line to render the display (doesn't work for Jupyter Notebook)
            
            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, axis=0)
            
            policy = model(state)
            
            action = np.random.choice(num_actions, 1, p=np.squeeze(policy))[0]
            action_prob = policy[0, action]  # probability of the action taken
            
            state, reward, done, _ = env.step(action)
            
            # collect samples
            rewards_history += reward,  # appends a variable to the list
            action_probs_history += action_prob,
            
            episode_reward += reward
            
            if done:
                break
                
        # Update running reward to check condition for solving
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward
        
        # Calculate expected value from rewards
        # - At each timestep what was the total reward received after that timestep
        # - Rewards in the past are discounted by multiplying them with gamma
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)
            
        # Normalize
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()

        # compute the loss
        
        action_probs_history = tf.convert_to_tensor(action_probs_history)
        cross_entropy = -tf.math.log(action_probs_history + 1e-6)
        loss = tf.reduce_sum(returns * cross_entropy)

        gradients = tape.gradient(loss, model.trainable_variables)  # compute the gradients of loss w.r.t model parameters
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # update the parameters accordingly
        action_probs_history, rewards_history = [], []  # clear the samples
        
    # Log details
    episode_count += 1
    if episode_count % 10 == 0:
        template = "running reward: {:.2f} at episode {}"
        print(template.format(running_reward, episode_count))

    if running_reward > 195:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

running reward: 11.74 at episode 10
running reward: 23.98 at episode 20
running reward: 28.89 at episode 30
running reward: 32.15 at episode 40
running reward: 37.01 at episode 50
running reward: 42.13 at episode 60
running reward: 55.84 at episode 70
running reward: 87.93 at episode 80
running reward: 102.97 at episode 90
running reward: 133.17 at episode 100
running reward: 154.64 at episode 110
running reward: 172.84 at episode 120
running reward: 183.74 at episode 130
running reward: 187.33 at episode 140
running reward: 179.81 at episode 150
running reward: 186.53 at episode 160
running reward: 191.94 at episode 170
running reward: 195.17 at episode 180
Solved at episode 180!
